<div style="direction:rtl; text-align:right">

# المشروع الثالث: هل تستطيع توقع إيجارات المنازل؟

<div style="direction:rtl; text-align:right">
أهلاً بكم في المشروع الثالث من علم البيانات. في هذا المشروع، سنعمل معا على بيانات إيجارات المنازل بناء على الكثير من السمات وذلك في البيانات الموجودة في هذا الرابط من موقع كاقل:

- https://www.kaggle.com/c/house-prices-advanced-regression-techniques

<div style="direction:rtl; text-align:right">
ملاحظة: عليك تعديل الخانات التي يوجد فيها الكود أدناه قبل تسليم المشروع. جميع الخانات الأخرى يجب ان تبقى كما هي بدون أي 
تعديل.

```
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
```

<div style="direction:rtl; text-align:right">
سنقوم أولاً بإحضار المكتبات اللازمة

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<div style="direction:rtl; text-align:right">

# قراءة البيانات

<div style="direction:rtl; text-align:right">
بعد أن قمنا بتنزيل البيانات من موقع كاقل، سنقوم هنا بقراءة البيانات. ولاحظ أن لدينا بيانات تدريب وبيانات اختبار. سنقوم هنا بدمج البيانات مع بعض (مع عزل سعر المنزل، المتغير الذي علينا توقع قيمته لاحقا).

In [ ]:
train_data = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')

In [ ]:
train_ids = train_data['Id']

In [ ]:
test_data = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')

In [ ]:
test_ids = test_data['Id']

<div style="direction:rtl; text-align:right">
وهنا نقوم بعزل متغير سعر المنزل.. وبعد ذلك سنقوم بحذفه من بيانات التدريب.

In [ ]:
y = train_data['SalePrice']

In [ ]:
train_data.drop(columns='SalePrice', inplace=True)

In [ ]:
train_data.set_index('Id', inplace=True)

In [ ]:
test_data.set_index('Id', inplace=True)

<div style="direction:rtl; text-align:right">
وأخيرا سنقوم بدمج الجدولين باستخدام الأمر concat:

In [ ]:
data = pd.concat([train_data, test_data], axis=0)

In [ ]:
data.shape

In [ ]:
data.head()

<div style="direction:rtl; text-align:right">
في هذه البيانات، لدينا عدد ضخم من متغيرات التنبؤ. وسنقضي هذا المشروع في النظر إلى هذه البيانات وتنقيحها ومن ثم إنشاء نموذج للتنبؤ بسعر المنزل. 

<div style="direction:rtl; text-align:right">

# تجهيز البيانات

<div style="direction:rtl; text-align:right">
في البداية، لنكتشف المتغيرات وأنواعها. يمكنك العودة إلى ملف وصف البيانات لقراءة وصف كل متغير.

In [ ]:
data.info()

<div style="direction:rtl; text-align:right">
يبدو أن لدينا خليط بين المتغيرات الرقمية (int64 أو float64) والمتغيرات الفئوية (object). كما أن لدينا العديد من البيانات المفقودة. بشكل عام، لا ينصح التخلص من البيانات المفقودة لأن فقدان البيانات بحد ذاته قد يكون عاملاً مساهماً في توقع سعر المنزل.

<div style="direction:rtl; text-align:right">
لمعرفة مجموع الخانات المفقودة في كل متغير سنقوم باستخدام الأمر التالي:

In [ ]:
data.isnull().sum()

<div style="direction:rtl; text-align:right">
نظراً لوجود عدد كبير من المتغيرات نلاحظ أن الأمر السابق لم يقم بطبع جميع المتغيرات. كما نلاحظ أن هناك الكثير من المتغيرات التي لا يوجد لها أي بيانات مفقودة. في الأمر التالي، سنقوم بطباعة المتغيرات وعدد الخانات المفقودة فقط إن كان لديها خانات مفقودة:

In [ ]:
missing_data = data.isnull().sum()

In [ ]:
for k, v in missing_data.items():
    if v>0 : 
        print(k,':\t' , v)

<div style="direction:rtl; text-align:right">
نجد أن بعض المتغيرات لديها قيم مفقودة قليلة جدا بينما متغيرات أخرى لديها قيم كثيرة جدا. كما نرى أن بعض القيم مكررة مثل 159 و 82 أو قريبة جداً من بعض. هذه الملاحظات مهم جداً وضعها في الحسبان حتى نقرر هل نبقيها ام نحذفها؟ ربما هذه المتغيرات قادمة من منطقة محددة أو لها صفات محددة. ولكن لن نخوض في أي من هذه التفاصيل الآن.

<div style="direction:rtl; text-align:right">
نريد الآن أن نستخرج قائمتين من الأعمدة: القائمة الأولى فيها الأعمدة الفئوية (categorical_cols) بينما القائمة الأخرى تحتوي الأعمدة الرقمية. قم باستخراج الأعمدة في الخانات التالية:

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

# HINT:   you will use data.columns to extract all columns
#         Then press data.dtypes and see what you get
# You can also use    np.isin( )

categorical_cols = <CODE HERE>
numerical_cols = <CODE HERE>
categorical_cols

In [ ]:
assert len(categorical_cols) == 43, 'Wrong number of categorical columns'
assert len(numerical_cols) == 36, 'Wrong number of neumerical columns'
print('OK! You may proceed.')

<div style="direction:rtl; text-align:right">
سنبدأ بتجهيز الأعمدة الفئوية، نحتاج لأن نتصرف مع القيم الفارغة في الأعمدة الفئوية،
نحتاج دالة تعطينا أسماء الأعمدة الفئوية التي تحتوي قيم فارغة، قم بتعديل الدالة أدناه لتعطينا قائمة بهذه الأسماء

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

def extract_categorical_cols_with_nans(data):
    # you can use `categorical_cols` in here
    categorical_cols_w_nans = <CODE HERE>   
    return categorical_cols_w_nans

categorical_null = extract_categorical_cols_with_nans(data)

In [ ]:
assert len(categorical_null) == 23, 'Wrong number of null categorical columns'
print('OK! You may proceed.')

<div style="direction:rtl; text-align:right">
سنقوم بتقسيم الأعمدة الفئوية ذات القيم الفارغة لقسمين: قسم للحذف و قسم للتعديل
سنحذف الأعمدة التي تحوي قيم فارغة في أكثر من نصفها
و سنعوض القيم الفارغة في بقية الأعمدة  
سنستخرج أسماء الأعمدة التي يجب حذفها (القيم الفارغة فيها أكثر من النصف)
و الأعمدة التي يجب استبدال القيم الفارغة فيها (القيم الفارغة فيها أقل أو تساوي النصف)

In [ ]:
to_remove = []  # الأعمدة التي يجب حذفها
to_fillna = []  # الأعمدة التي يجب استبدال القيم الفارغة فيها

for col in categorical_null:
    
    if len(data) - data[col].isna().sum() < len(data) / 2:
        to_remove.append(col)
    else:
        to_fillna.append(col)

In [ ]:
assert len(to_remove) == 4, 'Wrong number of categorical columns'
assert len(to_fillna) == 19, 'Wrong number of categorical columns'
print('OK! You may proceed.')

<div style="direction:rtl; text-align:right">
بعد أن قمنا باستخراج الأعمدة، قم أولا بحذف الأعمدة التي يجب حذفها (to_remove)  

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
<CODE HERE>

<div style="direction:rtl; text-align:right">
قم الآن بتحديث قائمة المتغيرات الفئوية ( يجب ألا تحوي الأعمدة التي حذفتها)

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
<CODE HERE>

In [ ]:
assert len(categorical_cols) == 39, 'Wrong number of categorical columns'
print('OK! You may proceed.')

<div style="direction:rtl; text-align:right">
بالنسبة للأعمدة في متغير  
to_fillna  
قم بملء القيم الفارغة بالقيمة الأكثر تكرارًا في كل عمود
أو قم بتعويضها بقيمة مميزة بالنسبة لكل عمود

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

<CODE HERE>

<div style="direction:rtl; text-align:right">
سنتعامل أولاً مع المتغيرات الفئوية. بخصوص المتغيرات الفئوية، سنقوم بتحويلها إلى عدة أعمدة رقمية (1 أو 0  لكل عمود) تماماً كما فعلنا في المشروع الأول باستخدام الأمر pd.get_dummies. ولكن هذه المرة ستكتبها أنت داخل الأمر أدناه.

<div style="direction:rtl; text-align:right">
الآن سوف نستخدم الكود السابق ولكن داخل حلقة تكرارية (for) حتى تسهل العملية. عليك إكمال الكود أدناه ولكن لا تنس أن تستخدم الأمر drop لحذف العمود الأصلي.

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

def convert_categorical_to_dummies(df):
    # HINT: you use `categorical_cols` here
    # also: you should set dummy_na to TRUE (this is new)
    
    
    # extract dummy variables first (use dummy_na= True) to keep missing data marked
    # then add them to the end of original `df`
    # then drop the original variable from df (use inplace=True)


    <CODE HERE>
    
    
    return df


<div style="direction:rtl; text-align:right">
هنا سنقوم باستخدام الأمر الذي أنشاناه أعلاه:

In [ ]:
data = convert_categorical_to_dummies(data)

In [ ]:
assert data.shape[1] == 333 or data.shape[1] == 314, 'Wrong shape.'
print('Ok! You may proceed!')

In [ ]:
data.columns

<div style="direction:rtl; text-align:right">
نستطيع استخدام الأمر describe لوصف الأعمدة الرقمية:

In [ ]:
data[numerical_cols].describe()

<div style="direction:rtl; text-align:right">
نريد الاستعلام عن عدد المتغيرات الرقمية التي تحوي قيماً مفقودة وذلك لمعالجتها. في الخانة التالية، سنقوم بطباعة أسماء الأعمدة الرقمية ذات القيم المفقودة  (باستخدام numerical_cols). هل تستطيع أن تكمل المهمة ؟ يجب أن تكتب البرنامج  في الأمر أدناه.

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

def extract_numerical_cols_with_nans(data):
    # you can use `numerical_cols` in here
    numerical_cols_w_nans = <CODE HERE>  

    return numerical_cols_w_nans

numerical_cols_w_nans = extract_numerical_cols_with_nans(data) # save as a list

numerical_cols_w_nans

<div style="direction:rtl; text-align:right">
بعد تفحص البيانات المفقودة، وجدنا أن عدداً من الأعمدة الرقمية لديها بيانات مفقودة. والسؤال الآن هو كيف نتعامل مع البيانات المفقودة؟

In [ ]:
assert len(numerical_cols_w_nans)==11, 'Numerical Columns with NaNs are in the wrong shape!'

<div style="direction:rtl; text-align:right">
لنقم أولاً بتفحص توزيع الأعمدة لكل من هذه المتغيرات باستخدام المثال التالي:

In [ ]:
data['LotFrontage'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
في LotFrontage، نجد أن البيانات تتبع توزيعا طبيعياً باستثناء بعض القيم العالية جد في أقصى اليمين (بعد 300). وبالتالي تعويض البيانات المفقودة بالوسيط (Median) قد يبدو خياراً مناسباً. لاحظ أن هذا المتغير هو أكثر متغير لديه قيم مفقودة.

In [ ]:
data['BsmtFinSF1'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
نجد أن عدد كبير من البيانات لديه قيمة 0 بينما باقي القيم تتناقص كلما زادت المساحة. هنا نستطيع التعويض بقيمة فريدة للقيم المفقودة ولتكن -1.

In [ ]:
data['BsmtFinSF2'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
نجد أن عدد كبير من البيانات لديه قيمة 0 بينما باقي القيم بالكاد نراها. هنا نستطيع التعويض ب -1

In [ ]:
data['BsmtUnfSF'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
هنا نجد نمطاً مشابهاً ومتكرراً، وهو أن أغلب البيانات موزعة حول قيمة 0 ولكن باقي القيم تتناقص بعد ذلك مع ازدياد المساحة. هنا سنقوم بالتعويض ب -1.

In [ ]:
data['TotalBsmtSF'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
هنا نستطيع أن نرى توزيعاً طبيعياً واضحا بما يعني أننا نستطيع استخدام الوسيط الحسابي.

In [ ]:
data['BsmtFullBath'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
هذا المتغير يقيس عدد دورات المياه (الكاملة) في الدور الأرضي وكما نرى القيم حول 0 و 1 و 2. سنقوم باستخدام -1 للتعويض عن القيم المفقودة هنا.

In [ ]:
data['BsmtHalfBath'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
هذا المتغير يقيس عدد دورات المياه (التي يوجد فيها تواليت فقط) في الدور الأرضي وكما نرى القيم حول 0 و 1 و 2. سنقوم باستخدام -1 للتعويض عن القيم المفقودة هنا.

In [ ]:
data['GarageCars'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
عدد السيارات في القراج، ومن الواضح أننا سوف نستخدم -1 للتعويض عن القيم المفقودة هنا.

In [ ]:
data['GarageArea'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
مساحة القراج تتبع توزيعا شبه طبيعي. في الحقيقة من الصعب ترجيح خيار على آخر، لكن قيمة -1 للخانة المفقودة في هذا المتغير تبدو خياراً أسلم.

In [ ]:
data['MasVnrArea'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
هذا المتغير يحوي على 8 قيم مفقودة فقط، ولكن توزيع الأعداد مختلف تماماً حيث أغلب القيم عبارة عن قيمة 0. ماذا نفعل بالقيم المفقودة؟ التعويض بالمتوسط أو الوسيط لا معنى له في هذه الحالة. فهنا إما أن نضع القيمة المفقودة 0 أو نختار قيمة فريدة تعبر عن القيم المفقودة حتى تساهم في توقعات النموذج، مثل -1 (كما شرحنا في الدرس).

In [ ]:
data['GarageYrBlt'].plot.hist(bins=40)

<div style="direction:rtl; text-align:right">
متغير GarageYrBlt يحوي على 81 قيمة مفقودة. وبالنظر إلى توزيع السنوات نجد أن لدينا توزيع متعرج قليلاً وهنا يصعب استخدام المتوسط أو الوسيط. فليس أمامنا سوى اختيار قيمة فريدة تعبر عن القيم المفقودة ولتكن -1.

<div style="direction:rtl; text-align:right">
بناء على المقترحات أعلاه، الآن هو دورك في استبدال القيم المفقودة بالقيم الجديدة.

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

# YOUR CODE HERE.


<CODE HERE>

In [ ]:
assert (data==-1).sum().sum() == 191, 'Did you fill in missing values the right way ?'

In [ ]:
assert data.isnull().sum().sum() == 0, 'There are still NaNs! Task is not completed.'

<div style="direction:rtl; text-align:right">
بعد التعامل مع القيم المفقودة، قد يكون من المناسب أن نفكر ما إن كنا نريد تسوية البيانات أم لا. في الحقيقة أن تسوية البيانات قد لا تعني شيئاً لأن جميع المساحات بحسب الوصف مقاسة بنفس الوحدات وهي قدم مربع (square feet). لكن إن كنا في سيناريو وجدنا فيه أن المساحات تتبع وحدات مختلفة، ففي هذه الحالة علينا تسوية البيانات قبل إخضاعها لأي تحليل.

<div style="direction:rtl; text-align:right">

# نمذجة البيانات

<div style="direction:rtl; text-align:right">
سنقوم الآن بإنشاء نموذج للتنبؤ بأسعار المنزل من خلال البيانات الموجودة. في هذه المرة سوف نستخدام الغابات العشوائية. تستطيع قراءة المزيد عن الغابات العشوائية من خلال هذا الدرس الموجز:

- https://www.syr-res.com/article/11885.html

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

<div style="direction:rtl; text-align:right">
أسهل وأسرع طريقة لمعرفة أداء النموذج هي باستخدام الأمر cross_val_score حيث يقوم هذا الأمر بتقسيم البيانات إلى بيانات تدريب وبيانات اختار باستخدام التحقق المتقاطع ومن ثم إجراء النمذجة بدون أي حاجة للتدخل. بعد ذلك، سنقوم بطباعة معدل "الدرجات" لنعرف بشكل عام عن متوسط أداء النموذج قبل تطبيقه على بيانات جديدة. سنقوم باستخدام مربع الترابط (R^2) للتحقق من النموذج.

<div style="direction:rtl; text-align:right">
ولكن قبل تنفيذ التحقق المتقاطع، سنقوم أولا بعزل بيانات التدريب، ثم سنقوم بعزل متغير الاستجابة عن متغيرات التنبؤ وحفظ البيانات في متغيرين: X و y. 

In [ ]:
train_data = data.loc[train_ids] # this is X
test_data  = data.loc[test_ids]  # this is test data

In [ ]:
rf_model = RandomForestRegressor(n_estimators=50)
scores_RF = cross_val_score(rf_model, train_data, y, cv=5, scoring='r2')

In [ ]:
scores_RF.mean()

<div style="direction:rtl; text-align:right">
حصلنا على مربع ترابط يبلغ حوالي 85% هل حصلت على هذا الرقم ؟ هذا الرقم باختصار هو إشارة للعلاقة بين توقعات النموذج وبين الأسعار الحقيقية: كلما كانت القيم قريبة من بعض كلما زادت هذه القيمة. سنتعرف على أساليب تقييم النماذج في درس قادم.

<div style="direction:rtl; text-align:right">
إذا أردت الحصول على درجة أعلى فيمكنك القراءة عن هذا النموذج، ومعرفة كيف يمكن اختبار قيم مختلفة للمدخلات المختلفة. جرب مثلاً تغيير n_estimators إلى قيمة أكبر، هل تجد نفس القيمة؟ لقراءة المزيد عن الغابات العشوائية، ننصح بقراءة توثيق سايكت ليرن لها (ولجميع الخوارزميات الأخرى)

- https://scikit-learn.org/stable/modules/ensemble.html#forest

<div style="direction:rtl; text-align:right">

# تسليم المشروع

<div style="direction:rtl; text-align:right">
قبل تسليم المشروع في موقع كاقل، سوف نستخدم النموذج أعلاه لتوقع أسعار البيوت ومن ثم حفظ الملف الناتج لتسليمه إلى كاقل.

In [ ]:
rf_model.fit(train_data, y) # training model on full data

In [ ]:
test_data['SalePrice'] = rf_model.predict(test_data)

In [ ]:
test_data['SalePrice'].to_csv('./house_predictions_submission.csv', header=True)

<div style="direction:rtl; text-align:right">
والآن اذهب إلى موقع كاقل وقم برفع هذا الملف لترى قوة نموذجك.

- https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit